### Lib

In [ ]:
# common
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import time, gc, string, math
from tqdm.notebook import tqdm
import warnings
import shutil
from collections import defaultdict

# sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# LGBM
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
import lightgbm as lgb


In [ ]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

### Read DF

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col='row_id')
sub = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col='row-col')

In [ ]:
data.describe()

In [ ]:
for col in data.columns:
    foo = data[col].dropna()
    print(col, foo.mean(), foo.std(), foo.skew())

In [ ]:
from scipy.stats import skew
for _ in range(10):
    x = np.random.normal(0, 1, 1_000_000)
    print(x.mean(), x.std(), skew(x))

In [ ]:
x.mean()

In [ ]:
na_col_list = []
for col in data.columns:
    if data[col].isna().sum() != 0:
        na_col_list.append(col)

In [ ]:
for col in na_col_list:
    data[col+'isna'] = data[col].isna().astype('int')

All auc is 0.5

isna columns is not importance.

In [ ]:
for col in na_col_list:
    train = data.drop(col, axis=1)
    
    target_col = col+'isna'
    X_train = train.drop(target_col, axis=1)
    y_train = train[target_col]
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, shuffle=True, random_state=0)

    print("train {}".format(target_col))
    model = LGBMClassifier(n_estimators = 1000, learning_rate = 0.1, random_state=0, min_child_samples=90, num_leaves=150, max_bins=511, n_jobs=-1)
    model.fit(X_train, y_train, eval_metric="auc", eval_set=[(X_valid, y_valid)], callbacks=[lgb.early_stopping(stopping_rounds=30), lgb.log_evaluation(32)])
    lgb.plot_importance(model, max_num_features=20, title=col)